## 

In [3]:
# Copied over
import read_data
import numpy as np
from matplotlib import pyplot as plt
import copy

from scipy.cluster.vq import vq, kmeans, whiten


In [4]:
(calls, allbases, demands, times, calls_latlong) = read_data.populate_data()

allbases = np.delete(np.array(allbases), 2, 1) # Array of all the bases
demands = np.delete(np.array(demands), 2, 1)   # Array of all the demands
times = np.array(times)                        # Array of all the times
calls_latlong = np.array(calls_latlong)        # Array of location of the calls